# Overview

This lab has been adapted from the angr [motivating example](https://github.com/angr/angr-doc/tree/master/examples/fauxware). It shows the basic lifecycle and capabilities of the angr framework. 

Note this lab (and other notebooks running angr) should be run with the Python 3 kernel!

Look at fauxware.c! This is the source code for a "faux firmware" (@zardus really likes the puns) that's meant to be a simple representation of a firmware that can authenticate users but also has a backdoor - the backdoor is that anybody who provides the string "SOSNEAKY" as their password will be automatically authenticated.

In [1]:
# import the python system and angr libraries

import angr
import sys

In [2]:
# We can use this as a basic demonstration of using angr for symbolic execution. 
# First, we load the binary into an angr project.

p = angr.Project('/home/pac/Desktop/labs/lab7/fauxware/fauxware')

In [3]:
# Now, we want to construct a representation of symbolic program state.
# SimState objects are what angr manipulates when it symbolically executes
# binary code.
# The entry_state constructor generates a SimState that is a very generic
# representation of the possible program states at the program's entry
# point. There are more constructors, like blank_state, which constructs a
# "blank slate" state that specifies as little concrete data as possible,
# or full_init_state, which performs a slow and pedantic initialization of
# program state as it would execute through the dynamic loader.

state = p.factory.entry_state()

# Now, in order to manage the symbolic execution process from a very high
# level, we have a SimulationManager. SimulationManager is just collections
# of states with various tags attached with a number of convenient
# interfaces for managing them.

sm = p.factory.simulation_manager(state)

In [4]:
# Now, we begin execution. This will symbolically execute the program until
# we reach a branch statement for which both branches are satisfiable.

sm.run(until=lambda sm_: len(sm_.active) > 1)

# If you look at the C code, you see that the first "if" statement that the
# program can come across is comparing the result of the strcmp with the
# backdoor password. So, we have halted execution with two states, each of
# which has taken a different arm of that conditional branch. If you drop
# an IPython shell here and examine sm.active[n].solver.constraints
# you will see the encoding of the condition that was added to the state to
# constrain it to going down this path, instead of the other one. These are
# the constraints that will eventually be passed to our constraint solver
# (z3) to produce a set of concrete inputs satisfying them.

# As a matter of fact, we'll do that now.

input_0 = sm.active[0].posix.dumps(0)
input_1 = sm.active[1].posix.dumps(0)

WARNING | 2020-05-25 18:09:10,546 | angr.state_plugins.symbolic_memory | The program is accessing memory or registers with an unspecified value. This could indicate unwanted behavior.
WARNING | 2020-05-25 18:09:10,556 | angr.state_plugins.symbolic_memory | angr will cope with this by generating an unconstrained symbolic variable and continuing. You can resolve this by:
WARNING | 2020-05-25 18:09:10,563 | angr.state_plugins.symbolic_memory | 1) setting a value to the initial state
WARNING | 2020-05-25 18:09:10,565 | angr.state_plugins.symbolic_memory | 2) adding the state option ZERO_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to make unknown regions hold null
WARNING | 2020-05-25 18:09:10,566 | angr.state_plugins.symbolic_memory | 3) adding the state option SYMBOL_FILL_UNCONSTRAINED_{MEMORY_REGISTERS}, to suppress these messages.
WARNING | 2020-05-25 18:09:10,568 | angr.state_plugins.symbolic_memory | Filling memory at 0x7fffffffffefff8 with 40 unconstrained bytes referenced from 0x109d800 

In [5]:
# We have used a utility function on the state's posix plugin to perform a
# quick and dirty concretization of the content in file descriptor zero,
# stdin. One of these strings should contain the substring "SOSNEAKY"!

if b'SOSNEAKY' in input_0:
    analysis_result = input_0
else:
    analysis_result = input_1
    
print("Result: " + str(analysis_result))

with open("/home/pac/Desktop/labs/lab7/fauxware/analysis_result", "wb") as file:
    file.write(analysis_result)

Result: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00SOSNEAKY\x00'


In [6]:
# You should be able to run this script and pipe its output to fauxware and
# fauxware will authenticate you!

import os
command = "/home/pac/Desktop/labs/lab7/fauxware/fauxware < /home/pac/Desktop/labs/lab7/fauxware/analysis_result"
print(os.popen(command).read())

Username: 
Password: 
Welcome to the admin console, trusted user!

